In [26]:
import sys
sys.path.append("../Scripts/preprocess.py")

ModuleNotFoundError: No module named 'preprocess'

In [1]:
import os

train_images_path = "../Data/Train/"
image_folders = os.listdir(train_images_path)

for folder in image_folders:
    image_files = os.listdir(train_images_path+folder+"/")
    i = 0
    for image_file in image_files:
        ## os.rename(train_images_path+folder+"/"+image_file, train_images_path+folder+"/"+ folder + "_" + str(i) + ".jpg")
        i +=1

In [85]:
import os

test_images_path = "../Data/Test/"
image_folders = os.listdir(test_images_path)

for folder in image_folders:
    image_files = os.listdir(test_images_path+folder+"/")
    i = 0
    for image_file in image_files:
        os.rename(test_images_path+folder+"/"+image_file, test_images_path+folder+"/"+ folder + "_" + str(i) + ".jpg")
        i +=1

In [21]:
import pandas as pd

labels = []
img_paths = []
train_images_path = "../Data/Train/"
image_folders = os.listdir(train_images_path)

for folder in image_folders:
    image_files = os.listdir(train_images_path+folder+"/")

    for image_file in image_files:
        img_paths.append(folder +"/" + image_file)
        labels.append(folder)
        
df = pd.DataFrame(zip(img_paths, labels), columns=["Image", "Label"])
df.to_csv('train_data.csv', header=False, index=False)

In [34]:

from preprocess import create_csv_dataset
test_images_path = "../Data/Test/"
csv_name = "test_data"
csv_path = "../Data/"
create_csv_dataset(test_images_path, csv_name, csv_path)

ModuleNotFoundError: No module named 'preprocess'

In [5]:
import pandas as pd
from torch.utils.data import Dataset
import cv2 as cv
import numpy as np
import os

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

class ColorImageDataset(Dataset):
    def __init__(self, img_labels, dataset_dir, transform=None) -> None:
        self.img_labels = img_labels
        self.dataset_dir = dataset_dir
        self.transfor = transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, index) -> None:
        img_path = os.path.join(self.dataset_dir, self.img_labels.iloc[index, 0])
        label = self.img_labels.iloc[index, 1]
        img = cv.imread(img_path) # opencv reads in bgr order
        mu = np.mean(img.reshape(3, -1), axis=1, dtype=np.float64)/255 # take the mean in every chanel for the first RGB moment (normalized)
        img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV) 
        img_hsv = img_hsv.astype(np.float64)
        img_hsv[:, :, 0] = np.cos(np.pi * img_hsv[:, :, 0]/180)
        ## hist = cv.calcHist([img_hsv], channels=[0, 1], mask=None, histSize=[8, 8], ranges=[[-1, 1], [0, 256]])
        hist_h = np.histogram(img_hsv[:, :, 0], bins=8, range=[-1, 1])
        hist_s = np.histogram(img_hsv[:, :, 1], bins=8, range=[0, 256])

        hist_h = softmax(np.asarray(hist_h[0], dtype=np.float64))
        hist_s = softmax(np.asarray(hist_s[0], dtype=np.float64))
        return np.concatenate([mu, hist_h, hist_s]), label


        


In [6]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

label_file = "train_data.csv"
dataset_dir = "../Data/Train"
data = pd.read_csv(label_file)

train_df, valid_df = train_test_split(data, test_size=0.2, random_state=42)    

train_dataset = ColorImageDataset(img_labels=train_df, dataset_dir=dataset_dir)
valid_dataset = ColorImageDataset(img_labels=valid_df, dataset_dir=dataset_dir)



In [7]:
sample, label = valid_dataset[0]
print(sample)
print(label)

sample, label = valid_dataset[1]
print(sample)
print(label)

[0.49232547 0.49545365 0.45116226 0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 1.        ]
Blue
[0.6226911  0.62285056 0.80863195 1.         0.         0.
 0.         0.         0.         0.         0.         1.
 0.         0.         0.         0.         0.         0.
 0.        ]
Violet


In [8]:
train_list_data = [data.tolist() for data, _ in train_dataset]
train_list_label = [label for _, label in train_dataset]


df_data = pd.DataFrame(train_list_data)
df_label = pd.DataFrame(train_list_label, columns=["Labels"])
df_data.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
count,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.00000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000
mean,0.584627,0.508739,0.518463,0.070588,0.082353,0.101961,0.05098,0.011765,0.023529,0.043137,0.615686,0.482353,0.039216,0.019608,0.035294,0.031373,0.043137,0.070588,0.278431
std,0.228585,0.211019,0.229275,0.256639,0.275442,0.303192,0.22039,0.108037,0.151876,0.203566,0.487389,0.500671,0.194489,0.138921,0.184885,0.174665,0.203566,0.256639,0.449108
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.431711,0.368204,0.356760,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.596592,0.512132,0.518394,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.772216,0.637077,0.686297,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,0.998625,0.998625,0.998625,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
valid_list_data = [data.tolist() for data, _ in valid_dataset]
valid_list_label = [label for _, label in valid_dataset]
print(train_list_data[0])

df_valid_data = pd.DataFrame(valid_list_data)
df_valid_label = pd.DataFrame(valid_list_label, columns=["Labels"])

[0.8078151621215091, 0.7666739209634347, 0.8053001254873104, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(df_data)
X_test = scaler.transform(df_valid_data)



In [24]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
count,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02,2.550000e+02
mean,-2.647120e-16,-9.404242e-17,3.970680e-16,3.483053e-18,3.483053e-17,4.005511e-17,-6.966105e-17,2.089832e-17,-3.831358e-17,6.966105e-18,-9.752547e-17,-7.488563e-17,-6.966105e-17,4.527968e-17,-3.483053e-18,5.572884e-17,1.393221e-17,4.876274e-17,-4.179663e-17
std,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00,1.001967e+00
min,-2.562615e+00,-2.415615e+00,-2.265765e+00,-2.755891e-01,-2.995723e-01,-3.369527e-01,-2.317736e-01,-1.091089e-01,-1.552301e-01,-2.123251e-01,-1.265718e+00,-9.653073e-01,-2.020305e-01,-1.414214e-01,-1.912730e-01,-1.799685e-01,-2.123251e-01,-2.755891e-01,-6.211840e-01
25%,-6.702798e-01,-6.672969e-01,-7.066661e-01,-2.755891e-01,-2.995723e-01,-3.369527e-01,-2.317736e-01,-1.091089e-01,-1.552301e-01,-2.123251e-01,-1.265718e+00,-9.653073e-01,-2.020305e-01,-1.414214e-01,-1.912730e-01,-1.799685e-01,-2.123251e-01,-2.755891e-01,-6.211840e-01
50%,5.244579e-02,1.611083e-02,-3.017329e-04,-2.755891e-01,-2.995723e-01,-3.369527e-01,-2.317736e-01,-1.091089e-01,-1.552301e-01,-2.123251e-01,7.900657e-01,-9.653073e-01,-2.020305e-01,-1.414214e-01,-1.912730e-01,-1.799685e-01,-2.123251e-01,-2.755891e-01,-6.211840e-01
75%,8.222677e-01,6.093762e-01,7.334634e-01,-2.755891e-01,-2.995723e-01,-3.369527e-01,-2.317736e-01,-1.091089e-01,-1.552301e-01,-2.123251e-01,7.900657e-01,1.035940e+00,-2.020305e-01,-1.414214e-01,-1.912730e-01,-1.799685e-01,-2.123251e-01,-2.755891e-01,1.609829e+00
max,1.814695e+00,2.326095e+00,2.098388e+00,3.628590e+00,3.338092e+00,2.967776e+00,4.314555e+00,9.165151e+00,6.442049e+00,4.709758e+00,7.900657e-01,1.035940e+00,4.949747e+00,7.071068e+00,5.228129e+00,5.556528e+00,4.709758e+00,3.628590e+00,1.609829e+00


In [10]:
from sklearn.svm import SVC


# import metrics to compute accuracy
from sklearn.metrics import accuracy_score


# instantiate classifier with default hyperparameters
svc=SVC(kernel='linear', C=10) 

print(df_label.head())
# fit classifier to training set
svc.fit(df_data,df_label["Labels"])



   Labels
0  Violet
1    Blue
2   Green
3  Violet
4  Yellow


SVC(C=10, kernel='linear')

In [11]:
y_pred=svc.predict(df_valid_data)


# compute and print accuracy score
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(df_valid_label, y_pred)))

Model accuracy score with default hyperparameters: 0.6406


In [73]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(5)
knn.fit(df_data,df_label["Labels"])

y_pred=knn.predict(df_valid_data)


# compute and print accuracy score
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(df_valid_label, y_pred)))

Model accuracy score with default hyperparameters: 0.5781


In [1]:
from datasets import ColorImageDataset

label_file = "../Data/test_data.csv"
test_data = pd.read_csv(label_file)
test_dataset = ColorImageDataset(img_labels=train_df, dataset_dir=dataset_dir)


ModuleNotFoundError: No module named 'datasets'

In [93]:
test_list_data = [data.tolist() for data, _ in test_dataset]
test_list_label = [label for _, label in test_dataset]

In [94]:
df_test_data = pd.DataFrame(test_list_data)
df_test_label = pd.DataFrame(test_list_label, columns=["Labels"])

In [95]:
y_pred=svc.predict(df_test_data)


# compute and print accuracy score
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(df_test_label, y_pred)))

Model accuracy score with default hyperparameters: 0.7176


In [113]:

pred=svc.predict(test_dataset[0][0].reshape(1, -1))
print(pred)
print(df_test_label.iloc[0][0])

['Violet']
Violet


In [3]:
# define a video capture object 
import cv2 as cv
vid = cv.VideoCapture(0) 
  
while(True): 
      
    # Capture the video frame 
    # by frame 
    ret, frame = vid.read() 
  
    # Display the resulting frame 
    cv.imshow('frame', frame) 
      
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv.waitKey(1) & 0xFF == ord('q'): 
        break
  
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv.destroyAllWindows() 